# Inferring ancetries

In [64]:
!gcloud auth application-default login

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=764086051850-6qr4p6gpi6hn506pt8ejuq83di341hur.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8085%2F&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth&state=xTb7G89OASvPaoslA56ArUBERgOisJ&access_type=offline&code_challenge=LeTyc2CrEsyoTY_cv5FmftXsOCjhiWf1hodsIpnyIvE&code_challenge_method=S256


Credentials saved to file: [/Users/yt4/.config/gcloud/application_default_credentials.json]

These credentials will be used by any library that requests Application Default Credentials (ADC).

Quota project "open-targets-genetics-dev" was added to ADC which can be used by Google client libraries for billing and quota. Note that some services may still bill the project owning

In [1]:
import os

import hail as hl
import numpy as np
import pandas as pd
import pyspark.sql.functions as f

pd.set_option("display.max_colwidth", None)
pd.set_option("display.expand_frame_repr", False)

from gentropy.common.session import Session
from gentropy.dataset.summary_statistics import SummaryStatistics

hail_dir = os.path.dirname(hl.__file__)
session = Session(hail_home=hail_dir, start_hail=True, extended_spark_conf={"spark.driver.memory": "12g",
    "spark.kryoserializer.buffer.max": "500m","spark.driver.maxResultSize":"3g"})
hl.init(sc=session.spark.sparkContext, log="/dev/null")

Loading BokehJS ...

24/07/11 09:24:44 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
pip-installed Hail requires additional configuration options in Spark referring
  to the path to the Hail Python module directory HAIL_DIR,
  e.g. /path/to/python/site-packages/hail:
    spark.jars=HAIL_DIR/backend/hail-all-spark.jar
    spark.driver.extraClassPath=HAIL_DIR/backend/hail-all-spark.jar
    spark.executor.extraClassPath=./hail-all-spark.jarRunning on Apache Spark version 3.3.4
SparkUI available at http://mib118093s.internal.sanger.ac.uk:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.127-bb535cd096c5
LOGGING: writing to /dev/null


In [2]:
from gentropy.dataset.study_index import StudyIndex
from gentropy.method.multiple_ancestries_ld import multipleAncestriesLD

In [3]:
gwasCat=StudyIndex.from_parquet(session,"gs://genetics_etl_python_playground/releases/24.06/study_index/gwas_catalog")

In [4]:
df=gwasCat.df
df=df.filter(f.col("hasSumstats")==True)

In [5]:
df.printSchema()

root
 |-- studyId: string (nullable = true)
 |-- projectId: string (nullable = true)
 |-- studyType: string (nullable = true)
 |-- traitFromSource: string (nullable = true)
 |-- traitFromSourceMappedIds: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- geneId: string (nullable = true)
 |-- biosampleFromSourceId: string (nullable = true)
 |-- pubmedId: string (nullable = true)
 |-- publicationTitle: string (nullable = true)
 |-- publicationFirstAuthor: string (nullable = true)
 |-- publicationDate: string (nullable = true)
 |-- publicationJournal: string (nullable = true)
 |-- backgroundTraitFromSourceMappedIds: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- initialSampleSize: string (nullable = true)
 |-- nCases: integer (nullable = true)
 |-- nControls: integer (nullable = true)
 |-- nSamples: integer (nullable = true)
 |-- cohorts: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- ldPopulationStructu

In [6]:
df=df.select("studyId","ldPopulationStructure")

In [7]:
from pyspark.sql.functions import size

df.withColumn("ldPopulationStructure_size", size(df["ldPopulationStructure"])).show()

+------------+---------------------+--------------------------+
|     studyId|ldPopulationStructure|ldPopulationStructure_size|
+------------+---------------------+--------------------------+
|  GCST001475|         [{nfe, 1.0}]|                         1|
|  GCST004634|         [{nfe, 1.0}]|                         1|
|  GCST004901| [{eas, 0.10175403...|                         2|
|  GCST006489|         [{nfe, 1.0}]|                         1|
|GCST90000583| [{eas, 0.5}, {nfe...|                         2|
|GCST90001818|         [{nfe, 1.0}]|                         1|
|GCST90004868|         [{nfe, 1.0}]|                         1|
|GCST90005485|         [{nfe, 1.0}]|                         1|
|GCST90005858|         [{nfe, 1.0}]|                         1|
|GCST90010780|         [{nfe, 1.0}]|                         1|
|GCST90011772|         [{eas, 1.0}]|                         1|
|GCST90018120|         [{nfe, 1.0}]|                         1|
|GCST90018593|         [{eas, 1.0}]|    

In [8]:
df=df.withColumn("ldPopulationStructure_size", size(df["ldPopulationStructure"]))

In [9]:
df_0=df

In [10]:
df.show()

+------------+---------------------+--------------------------+
|     studyId|ldPopulationStructure|ldPopulationStructure_size|
+------------+---------------------+--------------------------+
|  GCST001475|         [{nfe, 1.0}]|                         1|
|  GCST004634|         [{nfe, 1.0}]|                         1|
|  GCST004901| [{eas, 0.10175403...|                         2|
|  GCST006489|         [{nfe, 1.0}]|                         1|
|GCST90000583| [{eas, 0.5}, {nfe...|                         2|
|GCST90001818|         [{nfe, 1.0}]|                         1|
|GCST90004868|         [{nfe, 1.0}]|                         1|
|GCST90005485|         [{nfe, 1.0}]|                         1|
|GCST90005858|         [{nfe, 1.0}]|                         1|
|GCST90010780|         [{nfe, 1.0}]|                         1|
|GCST90011772|         [{eas, 1.0}]|                         1|
|GCST90018120|         [{nfe, 1.0}]|                         1|
|GCST90018593|         [{eas, 1.0}]|    

In [11]:
from pyspark.sql.functions import expr

df1 = df.filter(expr("exists(ldPopulationStructure, x -> x.ldPopulation = 'eas')"))
df1=df1.filter(f.col("ldPopulationStructure_size")==1)

In [12]:
df_pd=df1.toPandas()

In [13]:
df_pd

,studyId,ldPopulationStructure,ldPopulationStructure_size
0,GCST90011772,"[(eas, 1.0)]",1
1,GCST90018593,"[(eas, 1.0)]",1
2,GCST90018692,"[(eas, 1.0)]",1
3,GCST90060653,"[(eas, 1.0)]",1
4,GCST90060657,"[(eas, 1.0)]",1
...,...,...,...
1514,GCST90202745,"[(eas, 1.0)]",1
1515,GCST90203158,"[(eas, 1.0)]",1
1516,GCST90203332,"[(eas, 1.0)]",1
1517,GCST90204019,"[(eas, 1.0)]",1


In [14]:
listOfAncestries=["nfe","afr","eas"]
path_to_ld_index="gs://genetics-portal-dev-analysis/yt4/ld_index_AFs"
full_list_of_ancestries = ["oth", "amr", "fin", "ami", "mid", "nfe", "sas", "asj", "eas", "afr"]

In [15]:
def gwas_cat_run(studyid:str):
    filtered_df = df_0.filter(f.col("studyId") == studyid)
    ld_pop_structure = filtered_df.select("ldPopulationStructure").first()[0]
    ld_pop_structure_values = [(row["ldPopulation"], row["relativeSampleSize"]) for row in ld_pop_structure]
    print(ld_pop_structure_values)

    gwas=SummaryStatistics.from_parquet(session,"gs://gwas_catalog_data/harmonised_summary_statistics/"+studyid+".parquet")
    N=gwas.df.count()
    print(N)

    x=multipleAncestriesLD.infer_ancestry_proportions(
    session=session,
    path_to_ld_index=path_to_ld_index,
    gwas=gwas,
    listOfAncestries=listOfAncestries,
    chromosomeInput="7",
    start=100_000_000,
    end=100_300_000,
    maf_threshold = 0.05,
    number_of_snps = 100,
    zero_af_substitution = 1e-4,
    z_threshold= 3,
    number_of_runs= 100,
    )
    #x=multipleAncestriesLD.run_few_loci(
    #session=session,
    #path_to_ld_index=path_to_ld_index,
    #gwas=gwas,
    #listOfAncestries=listOfAncestries,
    #maf_threshold = 0.05,
    #number_of_snps = 30,
    #zero_af_substitution = 1e-4,
    #z_threshold= 3,
    #number_of_runs= 10,
    #)
    print(x)

    return {"ld_pop_structure_values":ld_pop_structure_values,"N":N,"results":x}

# Actual run

In [16]:
y=gwas_cat_run("GCST90043759")

[('nfe', 1.0)]


11796992


2024-07-11 09:25:28.362 Hail: INFO: Table.join: renamed the following fields on the right to avoid name conflicts:
    'age_distribution' -> 'age_distribution_1'
    'freq_meta' -> 'freq_meta_1'
    'age_index_dict' -> 'age_index_dict_1'
    'faf_index_dict' -> 'faf_index_dict_1'
    'popmax_index_dict' -> 'popmax_index_dict_1'
    'rf' -> 'rf_1'
    'freq_index_dict' -> 'freq_index_dict_1'
2024-07-11 09:25:37.543 Hail: INFO: Ordering unsorted dataset with network shuffle
2024-07-11 09:25:44.916 Hail: INFO: Coerced sorted dataset
2024-07-11 09:25:49.640 Hail: INFO: Table.join: renamed the following fields on the right to avoid name conflicts:
    'age_distribution' -> 'age_distribution_1'
    'freq_meta' -> 'freq_meta_1'
    'age_index_dict' -> 'age_index_dict_1'
    'faf_index_dict' -> 'faf_index_dict_1'
    'popmax_index_dict' -> 'popmax_index_dict_1'
    'rf' -> 'rf_1'
    'freq_index_dict' -> 'freq_index_dict_1'
2024-07-11 09:25:56.311 Hail: INFO: Ordering unsorted dataset with net

  Ancestry         means  medians       ses
0      nfe  9.921731e-01      1.0  0.016909
1      afr  6.151201e-07      0.0  0.000006
2      eas  7.826329e-03      0.0  0.016909


In [17]:
y["results"]

,Ancestry,means,medians,ses
0,nfe,9.921731e-01,1.0,0.016909
1,afr,6.151201e-07,0.0,0.000006
2,eas,7.826329e-03,0.0,0.016909


In [18]:
y=gwas_cat_run("GCST90014304")

[('nfe', 0.84861244490409), ('afr', 0.15138755509590998)]


6199123


  Ancestry     means  medians       ses
0      nfe  0.645532  0.66909  0.102999
1      afr  0.354347  0.33091  0.103079
2      eas  0.000122  0.00000  0.001209


In [19]:
y["results"]

,Ancestry,means,medians,ses
0,nfe,0.645532,0.66909,0.102999
1,afr,0.354347,0.33091,0.103079
2,eas,0.000122,0.00000,0.001209


In [20]:
studyid="GCST90014304"
gwas=SummaryStatistics.from_parquet(session,"gs://gwas_catalog_data/harmonised_summary_statistics/"+studyid+".parquet")
N=gwas.df.count()
print(N)

x=multipleAncestriesLD.infer_ancestry_proportions(
session=session,
path_to_ld_index=path_to_ld_index,
gwas=gwas,
listOfAncestries=["nfe","afr"],
chromosomeInput="7",
start=100_000_000,
end=100_300_000,
maf_threshold = 0.01,
number_of_snps = 200,
zero_af_substitution = 1e-4,
z_threshold= 3,
number_of_runs= 100,
)

6199123


In [21]:
print(x)

  Ancestry     means   medians       ses
0      nfe  0.614774  0.619646  0.064665
1      afr  0.385226  0.380354  0.064665


In [22]:
y=gwas_cat_run("GCST90005858")

[('nfe', 1.0)]


31413616


  Ancestry     means  medians       ses
0      nfe  0.975853      1.0  0.036995
1      afr  0.000146      0.0  0.000859
2      eas  0.024001      0.0  0.037029


In [23]:
y["results"]

,Ancestry,means,medians,ses
0,nfe,0.975853,1.0,0.036995
1,afr,0.000146,0.0,0.000859
2,eas,0.024001,0.0,0.037029


In [24]:
y=gwas_cat_run("GCST005843")

[('eas', 0.10678324718612628), ('amr', 0.0024095454325877975), ('nfe', 0.8508342592145307), ('afr', 0.03997294816675523)]


7537579


  Ancestry     means   medians       ses
0      nfe  0.653515  0.652750  0.112433
1      afr  0.203035  0.190639  0.104179
2      eas  0.143450  0.121326  0.097842


In [25]:
y["results"]

,Ancestry,means,medians,ses
0,nfe,0.653515,0.652750,0.112433
1,afr,0.203035,0.190639,0.104179
2,eas,0.143450,0.121326,0.097842


In [26]:
y=gwas_cat_run("GCST90301362")

[('afr', 1.0)]


13906959


  Ancestry     means  medians       ses
0      nfe  0.005357      0.0  0.013844
1      afr  0.994489      1.0  0.013903
2      eas  0.000155      0.0  0.000871


In [27]:
y["results"]

,Ancestry,means,medians,ses
0,nfe,0.005357,0.0,0.013844
1,afr,0.994489,1.0,0.013903
2,eas,0.000155,0.0,0.000871


In [29]:
y=gwas_cat_run("GCST90060657")

[('eas', 1.0)]


6851572


  Ancestry     means   medians       ses
0      nfe  0.663907  0.621946  0.190706
1      afr  0.006183  0.005049  0.005169
2      eas  0.329910  0.375029  0.192355


In [30]:
y["results"]

,Ancestry,means,medians,ses
0,nfe,0.663907,0.621946,0.190706
1,afr,0.006183,0.005049,0.005169
2,eas,0.329910,0.375029,0.192355


In [41]:
y=gwas_cat_run("GCST010118")

[('eas', 1.0)]


11225594


  Ancestry     means   medians       ses
0      nfe  0.267561  0.346713  0.200289
1      afr  0.004204  0.000000  0.010918
2      eas  0.728235  0.649027  0.196562


In [42]:
y["results"]

,Ancestry,means,medians,ses
0,nfe,0.267561,0.346713,0.200289
1,afr,0.004204,0.000000,0.010918
2,eas,0.728235,0.649027,0.196562


# GCST90005858

In [22]:
studyid="GCST90005858"
gwas=SummaryStatistics.from_parquet(session,"gs://gwas_catalog_data/harmonised_summary_statistics/"+studyid+".parquet")
N=gwas.df.count()
print(N)

31413616


In [23]:
session=session
path_to_ld_index=path_to_ld_index
gwas=gwas
listOfAncestries=listOfAncestries
chromosomeInput="7"
start=100_000_000
end=100_300_000
maf_threshold = 0.05
number_of_snps = 20
zero_af_substitution = 1e-4
z_threshold= 3
number_of_runs= 100

In [24]:
out_dict = multipleAncestriesLD.prepare_data_frames(
    session=session,
    listOfAncestries=listOfAncestries,
    chromosomeInput=chromosomeInput,
    start=start,
    end=end,
    path_to_ld_index=path_to_ld_index,
    gwas=gwas,
    full_list_of_ancestries=full_list_of_ancestries,
    maf_threshold=maf_threshold,
    z_threshold=z_threshold,
    filter_snps=True,
)

In [25]:
out_dict

{'LD': {'nfe': array([[ 1.        ,  0.5002967 , -0.61939744, ..., -0.43232243,
           0.28135577,  0.74116737],
         [ 0.5002967 ,  1.        ,  0.09062015, ..., -0.07955373,
           0.31775446,  0.38759397],
         [-0.61939744,  0.09062015,  1.        , ...,  0.58671545,
           0.19947036, -0.46818884],
         ...,
         [-0.43232243, -0.07955373,  0.58671545, ...,  1.        ,
           0.53192247, -0.59675848],
         [ 0.28135577,  0.31775446,  0.19947036, ...,  0.53192247,
           1.        ,  0.30672691],
         [ 0.74116737,  0.38759397, -0.46818884, ..., -0.59675848,
           0.30672691,  1.        ]]),
  'afr': array([[ 1.        ,  0.22347586, -0.6638619 , ..., -0.27818064,
           0.14212083,  0.45051042],
         [ 0.22347586,  1.        ,  0.17905218, ...,  0.02288898,
           0.46693352,  0.16562102],
         [-0.6638619 ,  0.17905218,  1.        , ...,  0.42494443,
           0.15282814, -0.44626224],
         ...,
         [-0.2

In [43]:
maf_threshold = 0.05
number_of_snps = 50
zero_af_substitution = 1e-4
z_threshold= 3
number_of_runs= 100

In [44]:
from scipy.optimize import minimize

m_p = out_dict["ref_af"][listOfAncestries]
m_p = np.array(m_p)
Z = np.array(out_dict["ref_af"]["z"].tolist())

mask = (m_p >= maf_threshold) & (m_p <= 1 - maf_threshold)
rows = np.all(mask, axis=1)
rows = (np.abs(Z) <= z_threshold) & rows
ind = np.where(rows)[0]

initial_alphas = np.ones(len(listOfAncestries))
bounds = [(0, 1)] * len(listOfAncestries)  # Non-negative bounds for alphas

out: list[np.ndarray] = []
for _ in range(number_of_runs):
    ind_to_run = np.random.choice(ind, number_of_snps, replace=False)
    values = out_dict["LD"][listOfAncestries[0]][ind_to_run, :][
        :, ind_to_run
    ]
    np.fill_diagonal(values, 0)
    avg_squared = np.mean(np.square(values))
    np.fill_diagonal(values, 1)
    if avg_squared >= 0.1:
        result = minimize(
            multipleAncestriesLD.fun_for_opt,
            initial_alphas,
            args=(
                Z,
                out_dict["LD"],
                m_p,
                listOfAncestries,
                ind_to_run,
                zero_af_substitution,
            ),
            bounds=bounds,
        )
        out = [result.x / np.sum(result.x)] + out

transposed_arrays = np.transpose(out)
means = np.mean(transposed_arrays, axis=1)
medians = np.median(transposed_arrays, axis=1)
ses = np.sqrt(np.var(transposed_arrays, axis=1))
df = pd.DataFrame(
    {
        "Ancestry": listOfAncestries,
        "means": means,
        "medians": medians,
        "ses": ses,
    }
)

df

,Ancestry,means,medians,ses
0,nfe,0.957939,1.000000e+00,0.073811
1,afr,0.000154,0.000000e+00,0.001289
2,eas,0.041906,1.222709e-17,0.073880


In [47]:
len(out)

100

In [119]:
initial_alphas = np.ones(len(listOfAncestries))
bounds = [(0, 1)] * len(listOfAncestries)  # Non-negative bounds for alphas

out: list[np.ndarray] = []

In [120]:
out

[]

In [125]:
ind_to_run = np.random.choice(ind, 30, replace=False)
values = out_dict["LD"][listOfAncestries[0]][ind_to_run, :][
    :, ind_to_run
]
np.fill_diagonal(values, 0)
avg_squared = np.mean(np.square(values))
np.fill_diagonal(values, 1)

In [126]:
avg_squared

0.2568862495920882

In [127]:
from scipy.optimize import minimize

result = minimize(
    multipleAncestriesLD.fun_for_opt,
    initial_alphas,
    args=(
        Z,
        out_dict["LD"],
        m_p,
        listOfAncestries,
        ind_to_run,
        zero_af_substitution,
    ),
    bounds=bounds,
)

In [128]:
result

  message: CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL
  success: True
   status: 0
      fun: 4.918265983028123
        x: [ 1.000e+00  0.000e+00  0.000e+00]
      nit: 1
      jac: [-0.000e+00  3.691e+01  7.667e+00]
     nfev: 8
     njev: 2
 hess_inv: <3x3 LbfgsInvHessProduct with dtype=float64>

In [130]:
np.mean(np.abs(Z[ind_to_run]))

0.7545768203687533

In [21]:
out: list[np.ndarray] = []
for _ in range(40):
    ind_to_run = np.random.choice(ind, number_of_snps, replace=False)
    values = out_dict["LD"][listOfAncestries[0]][ind_to_run, :][
        :, ind_to_run
    ]
    np.fill_diagonal(values, 0)
    avg_squared = np.mean(np.square(values))
    np.fill_diagonal(values, 1)
    if avg_squared >= 0.1:
        result = minimize(
            multipleAncestriesLD.fun_for_opt,
            initial_alphas,
            args=(
                Z,
                out_dict["LD"],
                m_p,
                listOfAncestries,
                ind_to_run,
                zero_af_substitution,
            ),
            bounds=bounds,
        )
        out = [result.x / np.sum(result.x)] + out

NameError: name 'ind' is not defined